Declare all imports

In [2]:
import random
from collections import Counter
from src import LIBRARY_ROOT
from src.dataobjs.cluster import Clusters
from src.dataobjs.mention_data import MentionData
from src.dataobjs.topics import Topics
from src.utils.io_utils import write_mention_to_json
from src.utils.sqlite_utils import select_all_from_clusters, create_connection
from src.utils.string_utils import StringUtils


Measure Clusters and Mentions by type 

In [3]:
dev_mentions = MentionData.read_mentions_json_to_mentions_data_list(str(LIBRARY_ROOT) +
            '/resources/dataset_full/wec/dev/Event_gold_mentions_validated2.json')

test_mentions = MentionData.read_mentions_json_to_mentions_data_list(str(LIBRARY_ROOT) +
            '/resources/dataset_full/wec/test/Event_gold_mentions_validated2.json')

train_mentions = MentionData.read_mentions_json_to_mentions_data_list(str(LIBRARY_ROOT) +
            '/resources/dataset_full/wec/train/Event_gold_mentions_validated3.json')

all_mentions = test_mentions

found = False
final_mentions = list()
all_types = dict()
for mention in all_mentions:
    if mention.mention_type not in all_types:
        all_types[mention.mention_type] = dict()
    if mention.coref_chain not in all_types[mention.mention_type]:
        all_types[mention.mention_type][mention.coref_chain] = 0
    all_types[mention.mention_type][mention.coref_chain] += 1

for _type in all_types.keys():
    print(str(_type) + "-Clusters=" + str(len(all_types[_type])))
    print(str(_type) + "-Mentions=" + str(sum([clust for clust in all_types[_type].values()])))
print("Done-" + str(all_types))

Remove Sport events and clean noise from location, date and nationality
 

In [16]:
origin_mentions = MentionData.read_mentions_json_to_mentions_data_list(str(LIBRARY_ROOT) +
                       '/resources/dataset_full/wec/train/Event_gold_mentions_clean3.json')

print("before=" + str(len(origin_mentions)))
new_mention = list()
removed = 0
for mention in origin_mentions:
    if mention.mention_type == 6:
        continue
    if 'colspan' in mention.mention_context:
        continue
        
    if StringUtils.is_verb_phrase(mention.tokens_str):
        new_mention.append(mention)
    elif mention.mention_ner not in ["GPE", "LOC", "DATE", "NORP", "PERSON"]:
        new_mention.append(mention)

print("total mentions remove=" + str(len(origin_mentions) - len(new_mention)))
print("total mentions in split-" + str(len(new_mention)))
write_mention_to_json(str(LIBRARY_ROOT) + '/resources/dataset_full/wec/train/Event_gold_mentions_clean4.json', new_mention)
print("Process Done!")

REMOVE SINGLETONS


In [ ]:
topics_ = Topics()
topics_.create_from_file(
    str(LIBRARY_ROOT) + '/resources/dataset_full/wec/train/Event_gold_mentions_limit500_validated2.json', keep_order=True)
clusters = topics_.convert_to_clusters()

fin_mentions = list()
for ments_list in clusters.values():
    if len(ments_list) > 1:
        fin_mentions.extend(ments_list)

write_mention_to_json(
    str(LIBRARY_ROOT) + '/resources/dataset_full/wec/train/Event_gold_mentions_limit500.json', fin_mentions)


Sample clusters for agreement


In [ ]:
_event1 = str(LIBRARY_ROOT) + '/resources/dataset_full/wec/train_clusters_for_validation.json'
mentions = MentionData.read_mentions_json_to_mentions_data_list(_event1)
clusters = Clusters.from_mentions_to_gold_clusters(mentions)
print(str(len(mentions)))
print(str(len(clusters)))
selected_clust = random.sample(list(clusters.values()), 400)
flat_list = [item for sublist in selected_clust for item in sublist]
print(len(flat_list))

write_mention_to_json(str(LIBRARY_ROOT) + '/resources/dataset_full/gvc/test/GVC_All_gold_event_mentions.json', flat_list)

Create examples for manual validation (from unclean data)


In [4]:
valid_mentions = list()
valid1 = str(LIBRARY_ROOT) + '/resources/dataset_full/wec/dev/Event_gold_mentions_validated2.json'
valid2 = str(LIBRARY_ROOT) + '/resources/dataset_full/wec/test/Event_gold_mentions_validated2.json'
# all_mentions.extend(MentionData.read_mentions_json_to_mentions_data_list(valid1))
valid_mentions.extend(MentionData.read_mentions_json_to_mentions_data_list(valid2))
valid_clusters = Clusters.from_mentions_to_gold_clusters(valid_mentions)

non_valid_mentions = list()
not_valid1 = str(LIBRARY_ROOT) + '/resources/bkp/final_dataset_unclean/WEC_Dev_Full_Event_gold_mentions.json'
not_valid2 = str(LIBRARY_ROOT) + '/resources/bkp/final_dataset_unclean/WEC_Test_Full_Event_gold_mentions.json'
# all_mentions.extend(MentionData.read_mentions_json_to_mentions_data_list(not_valid1))
non_valid_mentions.extend(MentionData.read_mentions_json_to_mentions_data_list(not_valid2))
non_valid_cluters = Clusters.from_mentions_to_gold_clusters(non_valid_mentions)

final_mentions = list()
for cluster_id, mention_list in non_valid_cluters.items():
    if cluster_id not in valid_clusters:
        for mention in mention_list:
            if StringUtils.is_verb_phrase(mention.tokens_str):
                final_mentions.append(mention)
            elif mention.mention_ner not in ["GPE", "LOC", "DATE", "NORP", "PERSON"]:
                final_mentions.append(mention)

write_mention_to_json(str(LIBRARY_ROOT) + '/resources/bkp/final_dataset_unclean/WEC_All_Full_Event_gold_mentions_Not_valid.json', final_mentions)
print("Process Done!")

Remove Big clusters and see affect on dataset size


In [14]:
MAX_CLUST = 70

origin_mentions = MentionData.read_mentions_json_to_mentions_data_list(str(LIBRARY_ROOT) +
                       '/resources/dataset_full/wec/train/Event_gold_mentions_clean3.json')

print("before=" + str(len(origin_mentions)))
clusters = Clusters.from_mentions_to_gold_clusters(origin_mentions)
new_mentions = list()
clusters_num = 0
for clust_key, clust in clusters.items():
    if len(clust) <= MAX_CLUST:
        clusters_num += 1
        new_mentions.extend(clust)

print("Clust=" + str(clusters_num))
print("Ment=" + str(len(new_mentions)))


Counting number of mentions in cluster with same string (and correlation with coref title)


In [3]:
origin_mentions = MentionData.read_mentions_json_to_mentions_data_list(str(LIBRARY_ROOT) +
                       '/resources/dataset_full/wec/all/Event_gold_mentions_clean9.json')

clusters = Clusters.from_mentions_to_gold_clusters(origin_mentions)
ret_counts = list()
for clust in clusters.values():
    clust_count = Counter([mention.tokens_str.lower() for mention in clust])
    ret_counts.append(clust[0].coref_link + "(" + str(len(clust)) + ")" + "=" + str(clust_count) + "\n")

f = open(str(LIBRARY_ROOT) + '/reports/wec.txt','w')
f.writelines(ret_counts)
f.close()

print("Done!")

Adding mention link(coref page name) to mentions file

In [2]:
origin_mentions = MentionData.read_mentions_json_to_mentions_data_list(str(LIBRARY_ROOT) +
                       '/resources/dataset_full/wec/train/Event_gold_mentions_clean4.json')
connection = create_connection("/Users/aeirew/workspace/DataBase/EnWikiLinks_v11.db")
clusters = select_all_from_clusters(connection)

for mention in origin_mentions:
    if mention.coref_chain not in clusters:
        raise Exception("ERROR: No such coref_chain")
    
    mention.coref_link = clusters[mention.coref_chain]
    
write_mention_to_json(str(LIBRARY_ROOT) + '/resources/dataset_full/wec/train/Event_gold_mentions_clean5.jso', origin_mentions)

print("Done!")

Limit number of mentions with same string in cluster to MAX 

In [8]:
MAX = 5
origin_mentions = MentionData.read_mentions_json_to_mentions_data_list(str(LIBRARY_ROOT) +
                       '/resources/dataset_full/wec/train/Event_gold_mentions_clean10_uncut.json')

clusters = Clusters.from_mentions_to_gold_clusters(origin_mentions)
final_mentions = list()
for clust in clusters.values():
    clust_dict_names = dict()
    for mention in clust:
        if mention.tokens_str.lower() not in clust_dict_names:
            clust_dict_names[mention.tokens_str.lower()] = list()
        clust_dict_names[mention.tokens_str.lower()].append(mention)
        
    for ment_list in clust_dict_names.values():
        if len(ment_list) >= MAX:
            final_mentions.extend(list(random.sample(ment_list, MAX)))
        else:
            final_mentions.extend(ment_list)

print("total mentions remove=" + str(len(origin_mentions) - len(final_mentions)))
print("total mentions in split-" + str(len(final_mentions)))
write_mention_to_json(str(LIBRARY_ROOT) + '/resources/dataset_full/wec/train/Event_gold_mentions_clean11.json', final_mentions)            
print("Done!")

total mentions remove=54268
total mentions in split-40622
Done!


Filter mentions with span longer then MAX_SPAN

In [7]:
MAX_SPAN = 7

origin_mentions = MentionData.read_mentions_json_to_mentions_data_list(str(LIBRARY_ROOT) +
                       '/resources/dataset_full/wec/train/Event_gold_mentions_clean7_uncut.json')

final_mentions = list()
for mention in origin_mentions:
    if (mention.tokens_number[-1] - mention.tokens_number[0]) <= MAX_SPAN:
        final_mentions.append(mention)
    
print("total mentions remove=" + str(len(origin_mentions) - len(final_mentions)))
print("total mentions in split-" + str(len(final_mentions)))
write_mention_to_json(str(LIBRARY_ROOT) + '/resources/dataset_full/wec/train/Event_gold_mentions_clean8_uncut.json', final_mentions)            
print("Done!")

Fix mention topic_id and mention_id

In [2]:
origin_mentions = MentionData.read_mentions_json_to_mentions_data_list(str(LIBRARY_ROOT) +
                       '/resources/dataset_full/wec/train/Event_gold_mentions_clean5_uncut.json')

index = 1
for mention in origin_mentions:
    mention.topic_id = -1
    mention.mention_id = index
    index += 1
    
write_mention_to_json(str(LIBRARY_ROOT) + '/resources/dataset_full/wec/train/Event_gold_mentions_clean6_uncut.json', origin_mentions)            
print("Done!")

Limit mention spans of verbs only to the VERB tokens

In [2]:
origin_mentions = MentionData.read_mentions_json_to_mentions_data_list(str(LIBRARY_ROOT) +
                       '/resources/dataset_full/wec/all/Event_gold_mentions_clean8_uncut_span7.json')

modified = 0
for mention in origin_mentions:
    if len(mention.tokens_number) > 1:
        phrase_index = StringUtils.extract_verb_phrase_index(mention)
        if len(phrase_index) > 0:
            modified += 1
            print("Before=" + mention.tokens_str)
            new_ment = " ".join(mention.mention_context[phrase_index[0]:phrase_index[-1] + 1]).strip()
            if len(new_ment) > 0:
                mention.tokens_str = new_ment
                mention.tokens_number = phrase_index
                print("After=" + mention.tokens_str)

print("Modified=" + str(modified))
write_mention_to_json(str(LIBRARY_ROOT) + '/resources/dataset_full/wec/all/Event_gold_mentions_clean9_uncut_verb.json', origin_mentions)            
print("Done!")

Remove mentions when context containin HTTP

In [2]:
origin_mentions = MentionData.read_mentions_json_to_mentions_data_list(str(LIBRARY_ROOT) +
                       '/resources/dataset_full/wec/train/Event_gold_mentions_clean6_uncut.json')
final_mentions = list()
for mention in origin_mentions:
    contx = " ".join(mention.mention_context)
    if "http" in contx:
        continue
    final_mentions.append(mention)
    
print("total mentions remove=" + str(len(origin_mentions) - len(final_mentions)))
print("total mentions in split-" + str(len(final_mentions)))
write_mention_to_json(str(LIBRARY_ROOT) + '/resources/dataset_full/wec/train/Event_gold_mentions_clean7.json', final_mentions)            
print("Done!")
